# VCG

Assume $n = 4$ items and $m = 5$ bidders. We solve the VCG mechanism for the most general (and harder) case of combinatorial auctions, and non additive bidders' evalutions. The other cases are easier case of this general case and will follow directly.

In [249]:
# Bidders' evaluations
v = {
    'a': {
        frozenset({'A'}):2, 
        frozenset({'A', 'B'}):5, 
        frozenset({'B'}):2,
        frozenset({'D'}):7
    },
    'b':{
        frozenset({'C'}): 10,
        frozenset({'D'}):8
    },
    'c':{
        frozenset({'C', 'D'}):15,
        frozenset({'D'}):10
    },
    'd':{
        frozenset({'B', 'C'}):10,
        frozenset({'B'}):8
    },
    'e':{
        frozenset({'A', 'B'}):11,
        frozenset({'A', 'B', 'C', 'D'}):20
    }
}

In [250]:
v

{'a': {frozenset({'A'}): 2,
  frozenset({'A', 'B'}): 5,
  frozenset({'B'}): 2,
  frozenset({'D'}): 7},
 'b': {frozenset({'C'}): 10, frozenset({'D'}): 8},
 'c': {frozenset({'C', 'D'}): 15, frozenset({'D'}): 10},
 'd': {frozenset({'B', 'C'}): 10, frozenset({'B'}): 8},
 'e': {frozenset({'A', 'B'}): 11, frozenset({'A', 'B', 'C', 'D'}): 20}}

In [251]:
import copy as cp

def from_dict_to_list_bids(bids):
    '''
    This function converts the dictionary of bidders' evaluation into a list of tuples
    '''
    l = [(bidder, itemset, value) for bidder in bids for itemset,value in bids[bidder].items()]
    return l

def is_admissible(configuration):
    '''
    This function returns True if the configuration is admissible and False otherwise.

    A configuration is admissible if and only if the following two conditions are satisfied:
        1) every bidder gets at most one bundle
        2) the allocated bundles are disjoint
    '''
    one_bundle_per_bidder = len(set([elem[0] for elem in configuration])) == len(configuration)
    disjoint = pairwise_disjoint([elem[1] for elem in configuration])
    return disjoint and one_bundle_per_bidder 

def pairwise_disjoint(sets):
    '''
    This function checks wether the sets contained in a list are pairwise disjoint
    '''
    union = set().union(*sets)
    return len(union) == sum(map(len, sets))


def binary_indices(n):
    '''
    Given an integer n, this function returns the indices of the bits that are set to one.
    It performs a conversion from decimal to binary number and it returns the indices of the bits equal to one.
    Such a function is useful to select the elements of a set in order to build all possible configurations.
    '''

    bits = []
    while n:
        bits.append(n % 2)
        n = n // 2
    return [i for i in range(len(bits)) if bits[i] == 1]


def compute_configuration_value(configuration):
    '''
    A configuration is a list of tuples with the following semantic: (bidder, itemset, value).
    This function computes the total value of a configuration.
    '''
    return sum([elem[2] for elem in configuration])

def compute_winner(bids):
    ''' 
    This function computes the winner of a combinatorial auction
    Determining the winner of a combinatorial auction, under the VCG mechanism amounts to solving the
    weighted set packing problem, shich is NP-Hard. Here, in order to solve this problem, we use an algorithm
    that enumerates all the possible assignments and selects the admissible assignments with the maximum
    social welfare.

    This implementation is not the most efficient possible, since it generates all the possible configurations
    including non admissible configurations. This problem can be mitigated through techniques such as 
    constraint propagation, recursive search strategies, and backtracking.
    '''
    l = from_dict_to_list_bids(bids)
    N = len(l)
    omega_star = []
    omega_star_value = 0
    for i in range(1, 2**N - 1):
        idx = binary_indices(i)
        configuration = [l[j] for j in idx]
        if is_admissible(configuration):
            value = compute_configuration_value(configuration)
            if value > omega_star_value:
                omega_star = configuration
                omega_star_value = value
    return (omega_star, omega_star_value)

def VCG_auction(bids):
    '''
    This function implements the VCG mechanism. It takes in input the bids for the auction and provides in output:
        1) the optimal allocation of the items
        2) the price each winner has to pay
    
    The format of the output is a list of tuples with the following semantic: (bidder, itemset, price).
    Example:
                ('a', {'I1', 'I2'}, 30) --> Bidder 'a' receives the bundle {'I1', 'I2'} and pays 30

    In VCG mechanism, the optimal allocation is the allocation which maximises the social welfare.That is:

                $$ \omega^* = \\argmax_{\omega \in \Omega} \sum_{i \in N} v_i(\omega) $$

    The amount each player pays is the externality they impose on all the other players. That is:

                $$ p_i(\omega^*) = \max_{\omega \in \Omega} \sum_{j \\neq i \in N} v_j(\omega) - \sum_{j \\neq i} v_j(w^*)$$ 

    '''
    omega_star, omega_star_value = compute_winner(bids)
    VCG_allocation = []
    for player in bids:
        bids_wo_player = cp.copy(bids)
        del bids_wo_player[player]
        _, value_wo_player = compute_winner(bids_wo_player)
        sum_omega_values = sum([elem[2] for elem in omega_star if elem[0] != player])
        itemset = [item[1] for item in omega_star if item[0] == player]
        VCG_payment = (player, itemset, value_wo_player - sum_omega_values)
        VCG_allocation.append(VCG_payment)
    return(VCG_allocation)

In [252]:
compute_winner(v)

([('b', frozenset({'C'}), 10),
  ('c', frozenset({'D'}), 10),
  ('e', frozenset({'A', 'B'}), 11)],
 31)

In [253]:
VCG_auction(v)

[('a', [], 0),
 ('b', [frozenset({'C'})], 5),
 ('c', [frozenset({'D'})], 7),
 ('d', [], 0),
 ('e', [frozenset({'A', 'B'})], 10)]